# TP - HMM
## Import


In [1]:
import numpy as np

## Variables

In [2]:
states = ("normal","confinement",'vacances')
obs =("sport","courses","sport","dormir")
emit_p = {
    'normal' :  {'dormir' : 0.4, 'sport' : 0.3, 'courses' : 0.3},
    'confinement' :  {'dormir' : 0.4, 'sport' : 0.2, 'courses' : 0.4},
    'vacances' : {'dormir' : 0.7, 'sport' : 0.1, 'courses' : 0.2}
}
start_p = {
    'normal' : 0.6,
    'confinement' : 0.1,
    'vacances' : 0.3
}
trans_p = {
    'normal' : {'normal' : 0.8, 'confinement' : 0.1, 'vacances' : 0.1},
    'confinement' : {'normal' : 0.3, 'confinement' : 0.6, 'vacances' : 0.1},
    'vacances' : {'normal' : 0.5, 'confinement' : 0.1, 'vacances' : 0.4}
}

## Algorithme

In [3]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    obs : array (T,)
        Observation state sequence. int dtype.
    start_p: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).
    trans_p : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    emit_p : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
    
    print ('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)
    return V,opt


def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%9d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)



In [4]:
viterbi(obs, states, start_p, trans_p, emit_p)

        0         1         2         3
normal: 0.18000 0.04320 0.01036 0.00331
confine: 0.02000 0.00720 0.00086 0.00041
vacance: 0.03000 0.00360 0.00043 0.00072
The steps of states are normal normal normal normal with highest probability of 0.0033177600000000003


([{'normal': {'prob': 0.18, 'prev': None},
   'confinement': {'prob': 0.020000000000000004, 'prev': None},
   'vacances': {'prob': 0.03, 'prev': None}},
  {'normal': {'prob': 0.043199999999999995, 'prev': 'normal'},
   'confinement': {'prob': 0.0072, 'prev': 'normal'},
   'vacances': {'prob': 0.0036, 'prev': 'normal'}},
  {'normal': {'prob': 0.010368, 'prev': 'normal'},
   'confinement': {'prob': 0.0008640000000000001, 'prev': 'normal'},
   'vacances': {'prob': 0.00043200000000000004, 'prev': 'normal'}},
  {'normal': {'prob': 0.0033177600000000003, 'prev': 'normal'},
   'confinement': {'prob': 0.00041472000000000004, 'prev': 'normal'},
   'vacances': {'prob': 0.00072576, 'prev': 'normal'}}],
 ['normal', 'normal', 'normal', 'normal'])